# 🌱 Ordinal Regression on Soil Arsenic Data

This notebook performs ordinal logistic regression to predict soil arsenic contamination levels (Non / Moderate / High) based on environmental variables.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# For ordinal logistic regression
from statsmodels.miscmodels.ordinal_model import OrderedModel

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Set plot style
sns.set(style="whitegrid")
%matplotlib inline

## 🔹 Step 1: Load Data

In [ ]:
url = "https://github.com/zia207/r-colab/raw/main/Data/Regression_analysis/bd_soil_arsenic.csv"
mf = pd.read_csv(url)

print("✅ Data loaded successfully!")
print(mf.head())

## 🔹 Step 2: Create Ordinal Class for Soil Arsenic (`SAs`)

In [ ]:
# Define bins and labels
bins = [-float('inf'), 14.8, 20, float('inf')]
labels = ['Non-contaminated', 'Moderately-contaminated', 'Highly-contaminated']

# Create ordinal categorical variable
mf['Class_As'] = pd.cut(mf['SAs'], bins=bins, labels=labels, ordered=True)

print("Distribution of contamination classes:")
print(mf['Class_As'].value_counts().sort_index())

## 🔹 Step 3: Data Processing

In [ ]:
# Select variables for modeling
selected_cols = [
    'WAs', 'WFe', 'SOC', 'SAoFe',
    'Year_Irrigation', 'Distance_STW',
    'Land_type', 'Class_As'
]

df = mf[selected_cols].copy()

# Convert categorical variables
df['Land_type'] = df['Land_type'].astype('category')
df['Class_As'] = df['Class_As'].astype('category')

# Normalize numerical columns (first 6)
numerical_cols = selected_cols[:6]
df[numerical_cols] = (df[numerical_cols] - df[numerical_cols].min()) / \
                     (df[numerical_cols].max() - df[numerical_cols].min())

print("✅ Processed ")
print(df.head())

## 🔹 Step 4: Stratified Train-Test Split

In [ ]:
np.random.seed(101)

# Create interaction for stratification
df['strata'] = df['Land_type'].astype(str) + "_" + df['Class_As'].astype(str)

train, test = train_test_split(
    df,
    test_size=0.3,
    stratify=df['strata'],
    random_state=101
)

train = train.drop(columns=['strata']).reset_index(drop=True)
test = test.drop(columns=['strata']).reset_index(drop=True)

print("Train Class_As proportions:")
print(train['Class_As'].value_counts(normalize=True))

print("\nTest Class_As proportions:")
print(test['Class_As'].value_counts(normalize=True))

## 🔹 Step 5: Prepare Features and Target

In [ ]:
feature_cols = [
    'WAs', 'WFe', 'SOC', 'SAoFe',
    'Year_Irrigation', 'Distance_STW',
    'Land_type'
]

# Target: ordinal encoding (0, 1, 2)
class_order = ['Non-contaminated', 'Moderately-contaminated', 'Highly-contaminated']
train['Class_As_ordinal'] = pd.Categorical(train['Class_As'], categories=class_order, ordered=True).codes
test['Class_As_ordinal'] = pd.Categorical(test['Class_As'], categories=class_order, ordered=True).codes

train_y = train['Class_As_ordinal']
test_y = test['Class_As_ordinal']

# Features
X_train = train[feature_cols]
X_test = test[feature_cols]

# One-hot encode Land_type
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("✅ Features and target are ready and aligned.")

## 🔹 Step 6: Fit Intercept-Only Model

In [ ]:
X_intercept = np.ones((X_train_df.shape[0], 1))

intercept_model = OrderedModel(
    endog=train_y,
    exog=X_intercept,
    distr='logit'
)

result_intercept = intercept_model.fit(method='bfgs', disp=False)
print(result_intercept.summary())

## 🔹 Step 7: Fit Full Model

In [ ]:
full_model = OrderedModel(
    endog=train_y,
    exog=X_train_df,
    distr='logit'
)

result_full = full_model.fit(method='bfgs', disp=False)
print(result_full.summary())

## 🔹 Step 8: Likelihood Ratio Test

In [ ]:
ll_full = result_full.llf
ll_intercept = result_intercept.llf
lr_stat = 2 * (ll_full - ll_intercept)
df_diff = len(result_full.params) - len(result_intercept.params)
p_value = 1 - stats.chi2.cdf(lr_stat, df_diff)

print("Likelihood Ratio Test")
print(f"LR Statistic: {lr_stat:.3f}")
print(f"p-value: {p_value:.2e}")

## 🔹 Step 9: Odds Ratios

In [ ]:
coef_names = X_train_df.columns
params = result_full.params[:len(coef_names)]
conf_int = result_full.conf_int().loc[coef_names]

or_vals = np.exp(params)
or_ci_lower = np.exp(conf_int.iloc[:, 0])
or_ci_upper = np.exp(conf_int.iloc[:, 1])

or_table = pd.DataFrame({
    'Odds Ratio': or_vals,
    '2.5%': or_ci_lower,
    '97.5%': or_ci_upper
}).round(3)

print("Odds Ratios:")
print(or_table)

## 🔹 Step 10: Model Performance (Train)

In [ ]:
pred_train_prob = result_full.predict()
pred_train_class = pred_train_prob.idxmax(axis=1)

acc_train = accuracy_score(train_y, pred_train_class)
print(f"Training Accuracy: {acc_train:.3f}")

cm_train = confusion_matrix(train_y, pred_train_class)
print("Confusion Matrix (Train):")
print(cm_train)

## 🔹 Step 11: Test Set Predictions

In [ ]:
pred_test_prob = result_full.predict(X_test_df)
pred_test_class = pred_test_prob.idxmax(axis=1)

acc_test = accuracy_score(test_y, pred_test_class)
print(f"Test Accuracy: {acc_test:.3f}")

cm_test = confusion_matrix(test_y, pred_test_class)
print("Confusion Matrix (Test):")
print(cm_test)

## 🔹 Step 12: Cross-Validation

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_accuracies = []

for train_idx, val_idx in skf.split(X_train_df, train_y):
    X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
    y_tr, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

    model_cv = OrderedModel(y_tr, X_tr, distr='logit').fit(method='bfgs', disp=False)
    pred = model_cv.predict(X_val).idxmax(axis=1)
    acc = accuracy_score(y_val, pred)
    cv_accuracies.append(acc)

avg_cv_acc = np.mean(cv_accuracies)
print(f"Cross-Validated Accuracy: {avg_cv_acc:.3f} (+/- {np.std(cv_accuracies)*2:.3f})")

## 🔹 Step 13: Visualize Odds Ratios

In [ ]:
or_table['Odds Ratio'].plot(
    kind='barh',
    xerr=[or_table['Odds Ratio'] - or_table['2.5%'],
          or_table['97.5%'] - or_table['Odds Ratio']],
    color='skyblue',
    edgecolor='black',
    title="Odds Ratios with 95% Confidence Intervals"
)
plt.axvline(x=1, color='red', linestyle='--', label='No effect (OR = 1)')
plt.xlabel("Odds Ratio")
plt.legend()
plt.tight_layout()
plt.show()

## 🔹 Step 14: Predicted Probabilities by `WAs`

In [ ]:
wa_grid = np.linspace(X_train_df['WAs'].min(), X_train_df['WAs'].max(), 50)
X_mean = X_train_df.mean().values.reshape(1, -1)
X_grid = np.tile(X_mean, (len(wa_grid), 1))
X_grid[:, X_train_df.columns.get_loc('WAs')] = wa_grid

pred_grid = result_full.predict(X_grid)

plt.figure(figsize=(8, 5))
for i, cls in enumerate(class_order):
    plt.plot(wa_grid, pred_grid[i], label=f"P({cls})")

plt.xlabel("Water As (WAs)")
plt.ylabel("Predicted Probability")
plt.title("Predicted Class Probabilities by Water As (WAs)")
plt.legend(title="Soil Arsenic Class")
plt.grid(False)
plt.tight_layout()
plt.show()